In [2]:
from pathlib import Path
import json
import pandas as pd
import numpy as np
import re
import ast
import pandas as pd

# Beam Experiment.

In [ ]:
# load in soft labels.
beam_search_2_2_40 = pd.read_csv("/dccstor/gma2/jhjenny9/search-and-learn/data/beam_numina/softlabels_2_2_40_beam_search_2_2_40.csv")
beam_search_2_4_40 = pd.read_csv("/dccstor/gma2/jhjenny9/search-and-learn/data/beam_numina/softlabels_2_4_40_beam_search_2_4_40.csv")
beam_search_4_2_40 = pd.read_csv("/dccstor/gma2/jhjenny9/search-and-learn/data/beam_numina/softlabels_4_2_40_beam_search_4_2_40.csv")
beam_search_4_4_40 = pd.read_csv("/dccstor/gma2/jhjenny9/search-and-learn/data/beam_numina/softlabels_4_4_40_Numina_beam_search_4_4_40_fifth_chunk.csv")
beam_search_8_2_40 = pd.read_csv("/dccstor/gma2/jhjenny9/search-and-learn/data/beam_numina/softlabels_8_2_40_beam_search_8_2_40.csv")
beam_search_8_4_20 = pd.read_csv("/dccstor/gma2/jhjenny9/search-and-learn/data/beam_numina/softlabels_8_4_20_beam_search_8_4_20.csv")
beam_search_8_4_40 = pd.read_csv("/dccstor/gma2/jhjenny9/search-and-learn/data/beam_numina/softlabels_beam_df_beam_search_8_4_40_chunk_28000_29000-beam_search_8_4_40_chunk_29000_30000.csv")
beam_search_16_2_40 = pd.read_csv("/dccstor/gma2/jhjenny9/search-and-learn/data/beam_numina/softlabels_16_2_40_beam_search_16_2_40.csv")
beam_search_16_4_20 = pd.read_csv("/dccstor/gma2/jhjenny9/search-and-learn/data/beam_numina/softlabels_16_4_20_beam_search_16_4_20.csv")
beam_search_16_4_40 = pd.read_csv("/dccstor/gma2/jhjenny9/search-and-learn/data/beam_numina/softlabels_16_4_40_beam_search_16_4_40.csv")
beam_search_32_2_20 = pd.read_csv("/dccstor/gma2/jhjenny9/search-and-learn/data/beam_numina/softlabels_32_2_20_beam_search_32_2_20.csv")
beam_search_32_4_40 = pd.read_csv("/dccstor/gma2/jhjenny9/search-and-learn/data/beam_numina/softlabels_32_4_40_beam_search_32_4_40.csv")

In [46]:
beam_search_2_2_40.head()

,problem,method,N,beam_width,chunk_size,sl
0,Consider the terms of an arithmetic sequence: ...,beam_search,2,2,40,1.000000
1,Suppose that $g(x) = 5x - 3$. What is $g^{-1}(...,beam_search,2,2,40,1.000000
2,A farmer has a rectangular field with dimensio...,beam_search,2,2,40,0.566667
3,Given the functions $f(x) = \log_a(1+x)$ and $...,beam_search,2,2,40,0.400000
4,Find all solutions to the equation $\displayst...,beam_search,2,2,40,0.000000


#### Get Latencies and Token-Counts

In [ ]:
import re
import pandas as pd
import os
import glob

def parse_beam_data_from_filename(file_path):
    # Extract beam size and question range from filename
    # Example: "Beam-8-5010-5020.out" -> beam_size=8, start=5010, end=5020
    
    with open(file_path, 'r') as f:
        content = f.read()
    
    # Split content into question blocks
    blocks = re.split(r'Total beam search time: \d+\.\d+ seconds\.', content)
    blocks = [block.strip() for block in blocks if block.strip()]
    
    data = []
    prev_total_tokens = 0
    
    for i, block in enumerate(blocks):
        # Extract the last "Total number of tokens generated thus far: X" before the beam search time
        token_matches = re.findall(r'Total number of tokens generated thus far: (\d+)', block)
        
        if token_matches:
            current_total_tokens = int(token_matches[-1])
            
            # Calculate tokens for this question (difference from previous)
            beam_tc = current_total_tokens - prev_total_tokens
            
            # Extract beam latency from the next block's start
            if i < len(blocks) - 1:
                block_start = content.find(block)
                block_end = block_start + len(block)
                
                time_match = re.search(r'Total beam search time: (\d+\.\d+) seconds\.', content[block_end:])
                if time_match:
                    beam_latency = float(time_match.group(1))
                else:
                    beam_latency = None
            else:
                # For the last block, look at the end of the file
                time_match = re.search(r'Total beam search time: (\d+\.\d+) seconds\.$', content)
                beam_latency = float(time_match.group(1)) if time_match else None
            
            data.append({
                'beam_tc': beam_tc,
                'beam_latency': beam_latency
            })
            prev_total_tokens = current_total_tokens
    
    return pd.DataFrame(data)

In [47]:
pita2240 = parse_beam_data_from_filename('/dccstor/gma2/jhjenny9/search-and-learn/data/beam_numina/pita2240.out')
pita2240.head()

,beam_tc,beam_latency
0,838,5.84
1,945,3.59
2,2999,13.43
3,2181,8.38
4,1989,8.75


In [ ]:
pita2240 = parse_beam_data_from_filename('/dccstor/gma2/jhjenny9/search-and-learn/data/beam_numina/pita2240.out')
pita4240 = parse_beam_data_from_filename('/dccstor/gma2/jhjenny9/search-and-learn/data/beam_numina/pita4240.out')
pita4420 = parse_beam_data_from_filename('/dccstor/gma2/jhjenny9/search-and-learn/data/beam_numina/pita4420.out')
pita8420 = parse_beam_data_from_filename('/dccstor/gma2/jhjenny9/search-and-learn/data/beam_numina/pita8420.out')
pita16240 = parse_beam_data_from_filename('/dccstor/gma2/jhjenny9/search-and-learn/data/beam_numina/pita16240.out')
pita16420 = parse_beam_data_from_filename('/dccstor/gma2/jhjenny9/search-and-learn/data/beam_numina/pita16420.out')
pita32220 = parse_beam_data_from_filename('/dccstor/gma2/jhjenny9/search-and-learn/data/beam_numina/pita32220.out')
pita32240 = parse_beam_data_from_filename('/dccstor/gma2/jhjenny9/search-and-learn/data/beam_numina/pita32240.out')


In [ ]:
import matplotlib.pyplot as plt

names = [
    "2x2_40", "4x2_40", "4x4_20", "8x4_20",
    "16x2_40", "16x4_20", "32x2_20", "32x2_40"
]
dfs = [
    pita2240, pita4240, pita4420, pita8420,
    pita16240, pita16420, pita32220, pita32240
]

mean_beam_tc = [df['beam_tc'].mean() for df in dfs]
mean_beam_latency = [df['beam_latency'].mean() for df in dfs]

fig, ax1 = plt.subplots(figsize=(10, 5))

ax1.set_xlabel('Config')
ax1.set_ylabel('Mean beam_tc', color='tab:blue')
ax1.plot(names, mean_beam_tc, marker='o', color='tab:blue', label='Mean beam_tc')
ax1.tick_params(axis='y', labelcolor='tab:blue')

ax2 = ax1.twinx()
ax2.set_ylabel('Mean beam_latency', color='tab:orange')
ax2.plot(names, mean_beam_latency, marker='s', color='tab:orange', label='Mean beam_latency')
ax2.tick_params(axis='y', labelcolor='tab:orange')

plt.title('Mean beam_tc and beam_latency by config')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [33]:
total_2240 = pd.concat([beam_search_2_2_40, pita2240], axis=1)
total_4240 = pd.concat([beam_search_4_2_40, pita4240], axis=1)
total_4420 = pd.concat([beam_search_4_4_40, pita4420], axis=1)
total_8420 = pd.concat([beam_search_8_4_20, pita8420], axis=1)
total_16240 = pd.concat([beam_search_16_2_40, pita16240], axis=1)
total_16420 = pd.concat([beam_search_16_4_20, pita16420], axis=1)
total_32220 = pd.concat([beam_search_32_2_20, pita32220], axis=1)
total_32240 = pd.concat([beam_search_32_4_40, pita32240], axis=1) # slight mismatch in beam width with this config.

all_beam_numina = pd.concat(
    [total_2240, total_4240, total_4420, total_8420, total_16240, total_16420, total_32220, total_32240],
    axis=0
)

all_beam_numina_df = pd.DataFrame(all_beam_numina)
# all_beam_numina_df.to_csv('/dccstor/gma2/jhjenny9/search-and-learn/data/beam_numina/all_beam_numina.csv', index=False)

In [54]:
all_beam_numina_df = all_beam_numina_df.rename(columns={'chunk_size': 'max_iteration'})
all_beam_numina_df.head(), all_beam_numina_df.shape

(                                             problem       method    N  \
 0  Consider the terms of an arithmetic sequence: ...  beam_search  2.0   
 1  Suppose that $g(x) = 5x - 3$. What is $g^{-1}(...  beam_search  2.0   
 2  A farmer has a rectangular field with dimensio...  beam_search  2.0   
 3  Given the functions $f(x) = \log_a(1+x)$ and $...  beam_search  2.0   
 4  Find all solutions to the equation $\displayst...  beam_search  2.0   
 
    beam_width  max_iteration        sl  beam_tc  beam_latency  
 0         2.0           40.0  1.000000      838          5.84  
 1         2.0           40.0  1.000000      945          3.59  
 2         2.0           40.0  0.566667     2999         13.43  
 3         2.0           40.0  0.400000     2181          8.38  
 4         2.0           40.0  0.000000     1989          8.75  ,
 (40000, 8))

In [58]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), '..'))
import data_processing.create_question_id

# Fit on filtered_df and save
q_to_id = data_processing.create_question_id.build_q_to_id(all_beam_numina_df, question_col="problem", start=0)
data_processing.create_question_id.save_mapping(q_to_id, "q_to_id_beam_numina.json")

# Load and apply to filtered_df
q_to_id = data_processing.create_question_id.load_mapping("q_to_id_beam_numina.json")
all_beam_numina_df_w_sb_idx = data_processing.create_question_id.apply_q_to_id(all_beam_numina_df, q_to_id, question_col="problem")
all_beam_numina_df_w_sb_idx.head()

,problem,method,N,beam_width,max_iteration,sl,beam_tc,beam_latency,sb_idx
0,Consider the terms of an arithmetic sequence: ...,beam_search,2.0,2.0,40.0,1.000000,838,5.84,1882
1,Suppose that $g(x) = 5x - 3$. What is $g^{-1}(...,beam_search,2.0,2.0,40.0,1.000000,945,3.59,8045
2,A farmer has a rectangular field with dimensio...,beam_search,2.0,2.0,40.0,0.566667,2999,13.43,495
3,Given the functions $f(x) = \log_a(1+x)$ and $...,beam_search,2.0,2.0,40.0,0.400000,2181,8.38,4300
4,Find all solutions to the equation $\displayst...,beam_search,2.0,2.0,40.0,0.000000,1989,8.75,2448


In [59]:
# create train/val/test split based on 'sb_idx' (query identifier)
from sklearn.model_selection import train_test_split

unique_sb_idx = all_beam_numina_df_w_sb_idx['sb_idx'].unique()

# Split into train and test only
sb_idx_train, sb_idx_test = train_test_split(
    unique_sb_idx,
    test_size=0.15,
    random_state=2
)

# Divide queries (represented by sb_idx) into train and test
df_train_beam_numina = all_beam_numina_df_w_sb_idx[all_beam_numina_df_w_sb_idx['sb_idx'].isin(sb_idx_train)].reset_index(drop=True)
df_test_beam_numina = all_beam_numina_df_w_sb_idx[all_beam_numina_df_w_sb_idx['sb_idx'].isin(sb_idx_test)].reset_index(drop=True)

df_train_beam_numina.to_csv("/dccstor/gma2/jhjenny9/search-and-learn/data/beam_numina/df_train_beam_numina.csv", index=False)
df_test_beam_numina.to_csv("/dccstor/gma2/jhjenny9/search-and-learn/data/beam_numina/df_test_beam_numina.csv", index=False)


In [3]:
import pandas as pd

df_train_beam_numina = pd.read_csv("/dccstor/gma2/jhjenny9/search-and-learn/data/beam_numina/df_train_beam_numina.csv")
df_test_beam_numina = pd.read_csv("/dccstor/gma2/jhjenny9/search-and-learn/data/beam_numina/df_test_beam_numina.csv")

In [4]:
df_train_beam_numina.head()

,problem,method,N,beam_width,max_iteration,sl,beam_tc,beam_latency,sb_idx
0,Consider the terms of an arithmetic sequence: ...,beam_search,2.0,2.0,40.0,1.000000,838,5.84,1882
1,Suppose that $g(x) = 5x - 3$. What is $g^{-1}(...,beam_search,2.0,2.0,40.0,1.000000,945,3.59,8045
2,A farmer has a rectangular field with dimensio...,beam_search,2.0,2.0,40.0,0.566667,2999,13.43,495
3,Find all solutions to the equation $\displayst...,beam_search,2.0,2.0,40.0,0.000000,1989,8.75,2448
4,"In $\triangle ABC$, the lengths of the sides o...",beam_search,2.0,2.0,40.0,0.000000,2311,6.95,5330


In [ ]:
import torch
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
from transformers import BertTokenizer, BertModel

print("CUDA available:", torch.cuda.is_available())
print("CUDA device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
local_model_path = "/u/jhjenny9/.cache/huggingface/models--bert-base-uncased/snapshots/86b5e0934494bd15c9632b12f734a8a67f723594"
tokenizer = BertTokenizer.from_pretrained(local_model_path)
model = BertModel.from_pretrained(local_model_path).to("cuda" if torch.cuda.is_available() else "cpu")
model.eval()
device = model.device  # Set device to match model
device
def get_cls_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
    cls_embedding = outputs.last_hidden_state[:, 0, :]  # CLS token is at position 0
    return cls_embedding.squeeze().numpy()

/dccstor/gma2/jhjenny9/envs/model_serve/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CUDA available: False
CUDA device name: No GPU


In [61]:
df_train_beam_numina.head()

,problem,method,N,beam_width,max_iteration,sl,beam_tc,beam_latency,sb_idx
0,Consider the terms of an arithmetic sequence: ...,beam_search,2.0,2.0,40.0,1.000000,838,5.84,1882
1,Suppose that $g(x) = 5x - 3$. What is $g^{-1}(...,beam_search,2.0,2.0,40.0,1.000000,945,3.59,8045
2,A farmer has a rectangular field with dimensio...,beam_search,2.0,2.0,40.0,0.566667,2999,13.43,495
3,Find all solutions to the equation $\displayst...,beam_search,2.0,2.0,40.0,0.000000,1989,8.75,2448
4,"In $\triangle ABC$, the lengths of the sides o...",beam_search,2.0,2.0,40.0,0.000000,2311,6.95,5330


In [ ]:
# (Training Data) Extract features: CLS + decoding parameters
cls_features = []
param_features = []
for _, row in df_train_beam_numina[:10000].iterrows():
    cls_vec = get_cls_embedding(row["problem"])
    beam_size = row["N"]
    beam_width = row["beam_width"]
    max_iteration = row["max_iteration"]
    #fifty_perc_beam_length = row["50%_beam_length"]
    num_samples = beam_size * beam_width
    
    # num_trees, num_samples, max_tokens
    features = np.concatenate([cls_vec, [beam_size, beam_width, max_iteration, num_samples]]) 
    cls_features.append(features)

X_train = np.array(cls_features)
y_train = df_train_beam_numina[:10000]["sl"].values.astype(int)

In [65]:
# (Testing Data) Extract features: CLS + decoding parameters
cls_features = []
param_features = []
for _, row in df_test_beam_numina[:1000].iterrows():
    cls_vec = get_cls_embedding(row["problem"])
    beam_size = row["N"]
    beam_width = row["beam_width"]
    max_iteration = row["max_iteration"]
    num_samples = beam_size * beam_width
    
    # num_trees, num_samples, max_tokens
    features = np.concatenate([cls_vec, [beam_size, beam_width, max_iteration, num_samples]]) 
    cls_features.append(features)
    
X_test = np.array(cls_features)
y_test = df_test_beam_numina[:1000]["sl"].values.astype(float)

In [ ]:
### Train MLP.
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report
# X_array = np.array(X)
# y_array = np.array(y)
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test) 

In [ ]:
# 2-layer MLP Binary Classifier
mlp_clf = MLPClassifier(hidden_layer_sizes=(128, 64), activation='relu', max_iter=500, random_state=42)
mlp_clf.fit(X_train, y_train)
y_pred = mlp_clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
y_proba = mlp_clf.predict_proba(X_test)[:, 1]
y_proba.shape
df_test['predicted_prob'] = y_proba
df_test.head(5)
df_test['predicted_prob']

#### Beam Times for Small vs. Large N Beam.

In [19]:
import re
import numpy as np

def extract_average_beam_search_time(log_file_path):
    """
    Extract all "Total beam search time:" values from a log file and calculate the average.
    
    Args:
        log_file_path (str): Path to the log file
        
    Returns:
        dict: Dictionary containing:
            - 'times': List of all beam search times in seconds
            - 'average': Average beam search time in seconds
            - 'count': Number of beam search operations
            - 'min': Minimum beam search time
            - 'max': Maximum beam search time
            - 'std': Standard deviation of beam search times
    """
    # Pattern to match "Total beam search time: X.XX seconds."
    pattern = r'Total beam search time: ([\d.]+) seconds\.'
    
    times = []
    
    try:
        with open(log_file_path, 'r') as file:
            for line in file:
                match = re.search(pattern, line)
                if match:
                    time_value = float(match.group(1))
                    times.append(time_value)
    except FileNotFoundError:
        print(f"Error: File '{log_file_path}' not found.")
        return None
    except Exception as e:
        print(f"Error reading file: {e}")
        return None
    
    if not times:
        print("No beam search times found in the log file.")
        return None
    
    # Calculate statistics
    times_array = np.array(times)
    result = {
        'times': times,
        'average': np.mean(times_array),
        'count': len(times),
        'min': np.min(times_array),
        'max': np.max(times_array),
        'std': np.std(times_array)
    }
    
    return result


In [20]:
# Test the function
result = extract_average_beam_search_time('/dccstor/gma2/jhjenny9/search-and-learn/large-beam-2.out')

print(f"Beam Time for N=128")
print(f"Found {result['count']} beam search operations")
print(f"Average time: {result['average']:.2f} seconds")
print(f"Min time: {result['min']:.2f} seconds")
print(f"Max time: {result['max']:.2f} seconds")
print(f"Standard deviation: {result['std']:.2f} seconds")
print(f"\nAll times: {result['times']}")

Beam Time for N=128
Found 10 beam search operations
Average time: 388.20 seconds
Min time: 133.41 seconds
Max time: 616.34 seconds
Standard deviation: 123.20 seconds

All times: [297.57, 403.46, 403.98, 415.98, 133.41, 616.34, 359.12, 514.49, 431.28, 306.37]


In [21]:
# Test the function
result = extract_average_beam_search_time('/dccstor/gma2/jhjenny9/search-and-learn/large-beam.out')

print(f"Beam Time for N=32")
print(f"Found {result['count']} beam search operations")
print(f"Average time: {result['average']:.2f} seconds")
print(f"Min time: {result['min']:.2f} seconds")
print(f"Max time: {result['max']:.2f} seconds")
print(f"Standard deviation: {result['std']:.2f} seconds")
print(f"\nAll times: {result['times']}")

Beam Time for N=32
Found 100 beam search operations
Average time: 81.80 seconds
Min time: 10.66 seconds
Max time: 264.00 seconds
Standard deviation: 57.81 seconds

All times: [24.76, 19.27, 99.12, 95.19, 26.03, 78.23, 13.39, 31.76, 134.76, 11.11, 56.41, 121.37, 109.11, 18.25, 264.0, 23.91, 62.78, 65.45, 64.4, 48.95, 118.92, 120.7, 24.39, 70.16, 160.42, 25.36, 167.64, 103.07, 88.3, 13.57, 27.39, 85.95, 21.11, 37.36, 41.51, 78.14, 77.44, 123.13, 122.26, 24.13, 20.46, 99.25, 21.17, 75.69, 24.67, 15.8, 19.05, 43.24, 115.7, 135.53, 67.32, 125.27, 73.35, 81.99, 224.16, 52.6, 103.65, 67.62, 130.81, 175.54, 21.22, 120.32, 176.65, 16.03, 105.63, 90.8, 23.85, 31.28, 36.32, 26.99, 91.97, 111.23, 217.35, 101.09, 35.78, 12.05, 51.53, 110.73, 189.84, 19.03, 11.21, 144.93, 16.41, 42.49, 98.03, 197.02, 102.67, 90.83, 139.4, 173.64, 77.79, 120.7, 82.18, 32.01, 40.53, 134.08, 104.03, 177.38, 222.48, 10.66]
